In [5]:
import math
import pickle
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from neuralprophet import NeuralProphet
from tqdm import tqdm, trange

from constants import (
    DEBUG_FORECAST_FOLDER,
    DEBUG_POSTGRESQL_PARQUET_TRAIN,
    DEBUG_POSTGRESQL_PARQUET_FUTURE,
    PG_LOG_DTYPES,
)
from forecast_metadata import ForecastMD
from generated_forecast_md import GeneratedForecastMD


In [6]:
pq_files = [Path(DEBUG_POSTGRESQL_PARQUET_TRAIN)]
print(f"Parquet files: {pq_files}")
for pq_file in tqdm(pq_files, desc="Reading Parquet files.", disable=True):
    df = pd.read_parquet(pq_file)
    df["log_time"] = df["log_time"].dt.tz_convert("UTC")
    print(f"{pq_file} has timestamps from {df['log_time'].min()} to {df['log_time'].max()}.")
    df["query_template"] = df["query_template"].replace("", np.nan)
    dropna_before = df.shape[0]
    df = df.dropna(subset=["query_template"])
    dropna_after = df.shape[0]
    print(
        f"Dropped {dropna_before - dropna_after} empty query template rows in {pq_file}. {dropna_after} rows remain."
    )
    
df.head()

Parquet files: [PosixPath('artifacts/tmp/data/train.parquet')]
artifacts/tmp/data/train.parquet has timestamps from 2022-05-19 02:10:25.683000+00:00 to 2022-05-19 02:10:44.468000+00:00.
Dropped 33 empty query template rows in artifacts/tmp/data/train.parquet. 121988 rows remain.


,log_time,session_id,session_line_num,virtual_transaction_id,transaction_id,query_template,query_params
29,2022-05-19 02:10:28.659000+00:00,6285a714.219b92,4,3/9,0,BEGIN,[]
30,2022-05-19 02:10:28.660000+00:00,6285a714.219b92,5,3/9,0,SET extra_float_digits = $1,['3']
31,2022-05-19 02:10:28.660000+00:00,6285a714.219b92,6,3/9,0,SET application_name = $1,['PostgreSQL JDBC Driver']
32,2022-05-19 02:10:28.661000+00:00,6285a714.219b92,7,3/9,0,COMMIT,[]
33,2022-05-19 02:10:28.677000+00:00,6285a714.219b92,8,3/10,0,select current_schema(),[]


In [49]:
vids = df.groupby("virtual_transaction_id").size()
vids = vids[(vids >= 5) & (vids <= 20)].index # vids with larger than 5 queries
vids[-20:]

Index(['3/97', '3/970', '3/973', '3/974', '3/976', '3/978', '3/98', '3/981',
       '3/984', '3/986', '3/987', '3/988', '3/989', '3/99', '3/990', '3/993',
       '3/994', '3/996', '3/997', '3/999'],
      dtype='object', name='virtual_transaction_id')

In [7]:
pd.options.display.max_colwidth = None
for i, vid in enumerate(vids):
  print(vid)
  if i >= 10:
    break
  display(df[df["virtual_transaction_id"] == vid][["query_template", "query_params"]])


NameError: name 'vids' is not defined

In [60]:
display(df[df["virtual_transaction_id"] == '3/103'][["query_template", "query_params"]])

,query_template,query_params
2056,BEGIN,[]
2057,"SELECT C_DISCOUNT, C_LAST, C_CREDIT FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_ID = $3","['1.0', '4.0', '155.0']"
2058,SELECT W_TAX FROM warehouse WHERE W_ID = $1,['1.0']
2059,"SELECT D_NEXT_O_ID, D_TAX FROM district WHERE D_W_ID = $1 AND D_ID = $2 FOR UPDATE","['1.0', '4.0']"
2060,UPDATE district SET D_NEXT_O_ID = D_NEXT_O_ID + $1 WHERE D_W_ID = $2 AND D_ID = $3,"['1', '1.0', '4.0']"
2061,"INSERT INTO oorder (O_ID, O_D_ID, O_W_ID, O_C_ID, O_ENTRY_D, O_OL_CNT, O_ALL_LOCAL) VALUES ($1, $2, $3, $4, $5, $6, $7)","['3003.0', '4.0', '1.0', '155.0', '2022-05-07 23:10:52.036', '6.0', '1.0']"
2062,"INSERT INTO new_order (NO_O_ID, NO_D_ID, NO_W_ID) VALUES ( $1, $2, $3)","['3003.0', '4.0', '1.0']"
2063,"SELECT I_PRICE, I_NAME , I_DATA FROM item WHERE I_ID = $1",['5158.0']
2064,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_02, S_DIST_03, S_DIST_04, S_DIST_05, S_DIST_06, S_DIST_07, S_DIST_08, S_DIST_09, S_DIST_10 FROM stock WHERE S_I_ID = $1 AND S_W_ID = $2 FOR UPDATE","['5158.0', '1.0']"
2065,"SELECT I_PRICE, I_NAME , I_DATA FROM item WHERE I_ID = $1",['71863.0']


In [8]:
from constants import DEBUG_FORECAST_FOLDER
import glob, os
from tqdm import tqdm
import pandas as pd

pq_files = glob.glob(os.path.join(DEBUG_FORECAST_FOLDER, "*.parquet"))

In [16]:
pd.options.display.max_colwidth = None

for pq_file in tqdm(pq_files, desc="Reading Parquet files.", disable=True):
    df = pd.read_parquet(pq_file)
    vids = df.groupby("session_id").size().index

    i = 0
    new_df = pd.DataFrame()
    for vid in vids:
        if i == 10:
            break
        df2 = df[df["session_id"] == vid][["query_template", "query_params"]]
        display(df2)
        if new_df.empty:
            new_df = df2
        else:
            new_df = pd.concat((new_df, df2))
        i += 1
    break

# new_df[:100]

,query_template,query_params
0,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
1,"SELECT C_DISCOUNT, C_LAST, C_CREDIT FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_ID = $3","{'$1': ''1'', '$2': ''8'', '$3': ''33'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
2,SELECT W_TAX FROM warehouse WHERE W_ID = $1,"{'$1': ''1'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3,"SELECT D_NEXT_O_ID, D_TAX FROM district WHERE D_W_ID = $1 AND D_ID = $2 FOR UPDATE","{'$1': ''1'', '$2': ''8'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
4,UPDATE district SET D_NEXT_O_ID = D_NEXT_O_ID + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''1'', '$2': ''1'', '$3': ''8'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
5,"INSERT INTO oorder (O_ID, O_D_ID, O_W_ID, O_C_ID, O_ENTRY_D, O_OL_CNT, O_ALL_LOCAL) VALUES ($1, $2, $3, $4, $5, $6, $7)","{'$1': ''3079'', '$2': ''8'', '$3': ''8'', '$4': ''33'', '$5': ''2022-05-19 02:10:43.417'', '$6': ''8'', '$7': ''1'', '$8': None, '$9': None}"
6,"INSERT INTO new_order (NO_O_ID, NO_D_ID, NO_W_ID) VALUES ( $1, $2, $3)","{'$1': ''3079'', '$2': ''8'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
7,"SELECT I_PRICE, I_NAME , I_DATA FROM item WHERE I_ID = $1","{'$1': ''68194'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
8,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_02, S_DIST_03, S_DIST_04, S_DIST_05, S_DIST_06, S_DIST_07, S_DIST_08, S_DIST_09, S_DIST_10 FROM stock WHERE S_I_ID = $1 AND S_W_ID = $2 FOR UPDATE","{'$1': ''68194'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
9,"SELECT I_PRICE, I_NAME , I_DATA FROM item WHERE I_ID = $1","{'$1': ''52617'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
44,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
45,UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2,"{'$1': ''871.11509164145'', '$2': ''20'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
46,"SELECT W_STREET_1, W_STREET_2, W_CITY, W_STATE, W_ZIP, W_NAME FROM warehouse WHERE W_ID = $1","{'$1': ''20'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
47,UPDATE district SET D_YTD = D_YTD + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''871.11509164145'', '$2': ''20'', '$3': ''2'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
48,"SELECT D_STREET_1, D_STREET_2, D_CITY, D_STATE, D_ZIP, D_NAME FROM district WHERE D_W_ID = $1 AND D_ID = $2","{'$1': ''20'', '$2': ''20'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
49,"SELECT C_FIRST, C_MIDDLE, C_ID, C_STREET_1, C_STREET_2, C_CITY, C_STATE, C_ZIP, C_PHONE, C_CREDIT, C_CREDIT_LIM, C_DISCOUNT, C_BALANCE, C_YTD_PAYMENT, C_PAYMENT_CNT, C_SINCE FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_LAST = $3 ORDER BY C_FIRST","{'$1': ''20'', '$2': ''3'', '$3': ''ABLEOUGHTPRI'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
50,"UPDATE customer SET C_BALANCE = -$1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6","{'$1': ''-2358.606966881747'', '$2': ''6719.315691976259'', '$3': ''20'', '$4': ''20'', '$5': ''3'', '$6': ''1019'', '$7': None, '$8': None, '$9': None}"
51,"INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)","{'$1': ''3'', '$2': ''20'', '$3': ''1019'', '$4': ''20'', '$5': ''20'', '$6': ''2022-05-19 02:10:43.598'', '$7': ''871.11509164145'', '$8': ''wstwxyo asqkuompj'', '$9': None}"
52,COMMIT,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
197,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
198,UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2,"{'$1': ''365.08339236147924'', '$2': ''9'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
199,"SELECT W_STREET_1, W_STREET_2, W_CITY, W_STATE, W_ZIP, W_NAME FROM warehouse WHERE W_ID = $1","{'$1': ''9'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
200,UPDATE district SET D_YTD = D_YTD + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''365.08339236147924'', '$2': ''9'', '$3': ''4'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
201,"SELECT D_STREET_1, D_STREET_2, D_CITY, D_STATE, D_ZIP, D_NAME FROM district WHERE D_W_ID = $1 AND D_ID = $2","{'$1': ''9'', '$2': ''9'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
202,"SELECT C_FIRST, C_MIDDLE, C_ID, C_STREET_1, C_STREET_2, C_CITY, C_STATE, C_ZIP, C_PHONE, C_CREDIT, C_CREDIT_LIM, C_DISCOUNT, C_BALANCE, C_YTD_PAYMENT, C_PAYMENT_CNT, C_SINCE FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_LAST = $3 ORDER BY C_FIRST","{'$1': ''9'', '$2': ''4'', '$3': ''ATIONPRESESE'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
203,"UPDATE customer SET C_BALANCE = -$1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6","{'$1': ''5116.828105046736'', '$2': ''2350.435636566832'', '$3': ''4'', '$4': ''9'', '$5': ''9'', '$6': ''224'', '$7': None, '$8': None, '$9': None}"
204,"INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)","{'$1': ''9'', '$2': ''9'', '$3': ''224'', '$4': ''9'', '$5': ''9'', '$6': ''2022-05-19 02:10:41.554'', '$7': ''365.08339236147924'', '$8': ''wstwxyo sbzxsibz'', '$9': None}"
205,COMMIT,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3260,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3261,SELECT NO_O_ID FROM new_order WHERE NO_D_ID = $1 AND NO_W_ID = $2 ORDER BY NO_O_ID ASC LIMIT $3,"{'$1': ''4'', '$2': ''1'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3262,DELETE FROM new_order WHERE NO_O_ID = $1 AND NO_D_ID = $2 AND NO_W_ID = $3,"{'$1': ''2140'', '$2': ''4'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3263,SELECT O_C_ID FROM oorder WHERE O_ID = $1 AND O_D_ID = $2 AND O_W_ID = $3,"{'$1': ''2140'', '$2': ''1'', '$3': ''4'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3264,UPDATE oorder SET O_CARRIER_ID = $1 WHERE O_ID = $2 AND O_D_ID = $3 AND O_W_ID = $4,"{'$1': ''1'', '$2': ''2140'', '$3': ''4'', '$4': ''4'', '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
...,...,...
3320,UPDATE oorder SET O_CARRIER_ID = $1 WHERE O_ID = $2 AND O_D_ID = $3 AND O_W_ID = $4,"{'$1': ''7'', '$2': ''2140'', '$3': ''1'', '$4': ''1'', '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3321,UPDATE order_line SET OL_DELIVERY_D = $1 WHERE OL_O_ID = $2 AND OL_D_ID = $3 AND OL_W_ID = $4,"{'$1': ''2022-05-19 02:10:42.611'', '$2': ''2140'', '$3': ''6'', '$4': ''1'', '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3322,SELECT SUM(OL_AMOUNT) AS OL_TOTAL FROM order_line WHERE OL_O_ID = $1 AND OL_D_ID = $2 AND OL_W_ID = $3,"{'$1': ''2140'', '$2': ''6'', '$3': ''4'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3323,"UPDATE customer SET C_BALANCE = C_BALANCE + $1, C_DELIVERY_CNT = C_DELIVERY_CNT + $2 WHERE C_W_ID = $3 AND C_D_ID = $4 AND C_ID = $5","{'$1': ''40154.104071585476'', '$2': ''1'', '$3': ''1'', '$4': ''6'', '$5': ''150'', '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3325,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3326,"SELECT C_DISCOUNT, C_LAST, C_CREDIT FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_ID = $3","{'$1': ''1'', '$2': ''9'', '$3': ''54'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3327,SELECT W_TAX FROM warehouse WHERE W_ID = $1,"{'$1': ''1'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3328,"SELECT D_NEXT_O_ID, D_TAX FROM district WHERE D_W_ID = $1 AND D_ID = $2 FOR UPDATE","{'$1': ''1'', '$2': ''9'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3329,UPDATE district SET D_NEXT_O_ID = D_NEXT_O_ID + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''1'', '$2': ''1'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
...,...,...
3389,"UPDATE stock SET S_QUANTITY = $1 , S_YTD = S_YTD + $2, S_ORDER_CNT = S_ORDER_CNT + $3, S_REMOTE_CNT = S_REMOTE_CNT + $4 WHERE S_I_ID = $5 AND S_W_ID = $6","{'$1': ''73'', '$2': ''1'', '$3': ''1'', '$4': ''0'', '$5': ''31873'', '$6': ''1'', '$7': None, '$8': None, '$9': None}"
3390,"UPDATE stock SET S_QUANTITY = $1 , S_YTD = S_YTD + $2, S_ORDER_CNT = S_ORDER_CNT + $3, S_REMOTE_CNT = S_REMOTE_CNT + $4 WHERE S_I_ID = $5 AND S_W_ID = $6","{'$1': ''21'', '$2': ''1'', '$3': ''1'', '$4': ''0'', '$5': ''31873'', '$6': ''1'', '$7': None, '$8': None, '$9': None}"
3391,"UPDATE stock SET S_QUANTITY = $1 , S_YTD = S_YTD + $2, S_ORDER_CNT = S_ORDER_CNT + $3, S_REMOTE_CNT = S_REMOTE_CNT + $4 WHERE S_I_ID = $5 AND S_W_ID = $6","{'$1': ''55'', '$2': ''1'', '$3': ''1'', '$4': ''0'', '$5': ''31873'', '$6': ''9'', '$7': None, '$8': None, '$9': None}"
3392,"UPDATE stock SET S_QUANTITY = $1 , S_YTD = S_YTD + $2, S_ORDER_CNT = S_ORDER_CNT + $3, S_REMOTE_CNT = S_REMOTE_CNT + $4 WHERE S_I_ID = $5 AND S_W_ID = $6","{'$1': ''98'', '$2': ''9.299494736906642'', '$3': ''1'', '$4': ''0'', '$5': ''31873'', '$6': ''1'', '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3394,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3395,UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2,"{'$1': ''4092.663592825597'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3396,"SELECT W_STREET_1, W_STREET_2, W_CITY, W_STATE, W_ZIP, W_NAME FROM warehouse WHERE W_ID = $1","{'$1': ''1'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3397,UPDATE district SET D_YTD = D_YTD + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''4092.663592825597'', '$2': ''1'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3398,"SELECT D_STREET_1, D_STREET_2, D_CITY, D_STATE, D_ZIP, D_NAME FROM district WHERE D_W_ID = $1 AND D_ID = $2","{'$1': ''1'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3399,"SELECT C_FIRST, C_MIDDLE, C_ID, C_STREET_1, C_STREET_2, C_CITY, C_STATE, C_ZIP, C_PHONE, C_CREDIT, C_CREDIT_LIM, C_DISCOUNT, C_BALANCE, C_YTD_PAYMENT, C_PAYMENT_CNT, C_SINCE FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_LAST = $3 ORDER BY C_FIRST","{'$1': ''1'', '$2': ''1'', '$3': ''PRESPRICALLY'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3400,"UPDATE customer SET C_BALANCE = -$1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6","{'$1': ''2154.295959439296'', '$2': ''4203.865809875997'', '$3': ''2'', '$4': ''1'', '$5': ''1'', '$6': ''6400'', '$7': None, '$8': None, '$9': None}"
3401,"INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)","{'$1': ''2'', '$2': ''1'', '$3': ''6400'', '$4': ''1'', '$5': ''1'', '$6': ''2022-05-19 02:10:42.993'', '$7': ''4092.663592825597'', '$8': ''wstwxyo edfwgm'', '$9': None}"
3402,COMMIT,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3403,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3404,UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2,"{'$1': ''2167.445678666747'', '$2': ''0'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3405,"SELECT W_STREET_1, W_STREET_2, W_CITY, W_STATE, W_ZIP, W_NAME FROM warehouse WHERE W_ID = $1","{'$1': ''0'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3406,UPDATE district SET D_YTD = D_YTD + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''2167.445678666747'', '$2': ''0'', '$3': ''9'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3407,"SELECT D_STREET_1, D_STREET_2, D_CITY, D_STATE, D_ZIP, D_NAME FROM district WHERE D_W_ID = $1 AND D_ID = $2","{'$1': ''0'', '$2': ''9'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3408,"SELECT C_FIRST, C_MIDDLE, C_ID, C_STREET_1, C_STREET_2, C_CITY, C_STATE, C_ZIP, C_PHONE, C_CREDIT, C_CREDIT_LIM, C_DISCOUNT, C_BALANCE, C_YTD_PAYMENT, C_PAYMENT_CNT, C_SINCE FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_LAST = $3 ORDER BY C_FIRST","{'$1': ''0'', '$2': ''9'', '$3': ''CALLYPRIPRES'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3409,"UPDATE customer SET C_BALANCE = $1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6","{'$1': ''37599.84622547856'', '$2': ''6156.066398946365'', '$3': ''2'', '$4': ''0'', '$5': ''9'', '$6': ''478'', '$7': None, '$8': None, '$9': None}"
3410,"INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)","{'$1': ''9'', '$2': ''0'', '$3': ''478'', '$4': ''9'', '$5': ''0'', '$6': ''2022-05-19 02:10:43.836'', '$7': ''2167.445678666747'', '$8': ''wstwxyo mcjzpbe'', '$9': None}"
3411,COMMIT,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3412,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3413,UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2,"{'$1': ''4155.953220352543'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3414,"SELECT W_STREET_1, W_STREET_2, W_CITY, W_STATE, W_ZIP, W_NAME FROM warehouse WHERE W_ID = $1","{'$1': ''1'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3415,UPDATE district SET D_YTD = D_YTD + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''4155.953220352543'', '$2': ''1'', '$3': ''3'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3416,"SELECT D_STREET_1, D_STREET_2, D_CITY, D_STATE, D_ZIP, D_NAME FROM district WHERE D_W_ID = $1 AND D_ID = $2","{'$1': ''1'', '$2': ''3'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3417,"SELECT C_FIRST, C_MIDDLE, C_ID, C_STREET_1, C_STREET_2, C_CITY, C_STATE, C_ZIP, C_PHONE, C_CREDIT, C_CREDIT_LIM, C_DISCOUNT, C_BALANCE, C_YTD_PAYMENT, C_PAYMENT_CNT, C_SINCE FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_LAST = $3 ORDER BY C_FIRST","{'$1': ''1'', '$2': ''10'', '$3': ''ATIONABLECALLY'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3418,"UPDATE customer SET C_BALANCE = -$1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6","{'$1': ''48.14486402559396'', '$2': ''2916.8498691573495'', '$3': ''10'', '$4': ''1'', '$5': ''3'', '$6': ''554'', '$7': None, '$8': None, '$9': None}"
3419,"INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)","{'$1': ''3'', '$2': ''1'', '$3': ''554'', '$4': ''3'', '$5': ''1'', '$6': ''2022-05-19 02:10:40.243'', '$7': ''4155.953220352543'', '$8': ''wstwxyo fuwaul'', '$9': None}"
3420,COMMIT,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3421,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3422,"SELECT C_DISCOUNT, C_LAST, C_CREDIT FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_ID = $3","{'$1': ''1'', '$2': ''1'', '$3': ''1869'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3423,SELECT W_TAX FROM warehouse WHERE W_ID = $1,"{'$1': ''1'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3424,"SELECT D_NEXT_O_ID, D_TAX FROM district WHERE D_W_ID = $1 AND D_ID = $2 FOR UPDATE","{'$1': ''1'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3425,UPDATE district SET D_NEXT_O_ID = D_NEXT_O_ID + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''-232'', '$2': ''1'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3426,"INSERT INTO oorder (O_ID, O_D_ID, O_W_ID, O_C_ID, O_ENTRY_D, O_OL_CNT, O_ALL_LOCAL) VALUES ($1, $2, $3, $4, $5, $6, $7)","{'$1': ''3061'', '$2': ''1'', '$3': ''1'', '$4': ''1869'', '$5': ''2022-05-19 02:10:37.807'', '$6': ''8'', '$7': ''1'', '$8': None, '$9': None}"
3427,"INSERT INTO new_order (NO_O_ID, NO_D_ID, NO_W_ID) VALUES ( $1, $2, $3)","{'$1': ''3061'', '$2': ''1'', '$3': ''1'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3428,"SELECT I_PRICE, I_NAME , I_DATA FROM item WHERE I_ID = $1","{'$1': ''45442'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3429,"SELECT S_QUANTITY, S_DATA, S_DIST_01, S_DIST_02, S_DIST_03, S_DIST_04, S_DIST_05, S_DIST_06, S_DIST_07, S_DIST_08, S_DIST_09, S_DIST_10 FROM stock WHERE S_I_ID = $1 AND S_W_ID = $2 FOR UPDATE","{'$1': ''45442'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3430,"SELECT I_PRICE, I_NAME , I_DATA FROM item WHERE I_ID = $1","{'$1': ''49900'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


,query_template,query_params
3453,BEGIN,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3454,UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2,"{'$1': ''1929.2926003643217'', '$2': ''1'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3455,"SELECT W_STREET_1, W_STREET_2, W_CITY, W_STATE, W_ZIP, W_NAME FROM warehouse WHERE W_ID = $1","{'$1': ''1'', '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3456,UPDATE district SET D_YTD = D_YTD + $1 WHERE D_W_ID = $2 AND D_ID = $3,"{'$1': ''1929.2926003643217'', '$2': ''1'', '$3': ''6'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3457,"SELECT D_STREET_1, D_STREET_2, D_CITY, D_STATE, D_ZIP, D_NAME FROM district WHERE D_W_ID = $1 AND D_ID = $2","{'$1': ''1'', '$2': ''6'', '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3458,"SELECT C_FIRST, C_MIDDLE, C_LAST, C_STREET_1, C_STREET_2, C_CITY, C_STATE, C_ZIP, C_PHONE, C_CREDIT, C_CREDIT_LIM, C_DISCOUNT, C_BALANCE, C_YTD_PAYMENT, C_PAYMENT_CNT, C_SINCE FROM customer WHERE C_W_ID = $1 AND C_D_ID = $2 AND C_ID = $3","{'$1': ''1'', '$2': ''6'', '$3': ''643'', '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"
3459,"UPDATE customer SET C_BALANCE = -$1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6","{'$1': ''2812.533312562009'', '$2': ''4435.752587185615'', '$3': ''2'', '$4': ''1'', '$5': ''1'', '$6': ''956'', '$7': None, '$8': None, '$9': None}"
3460,"INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)","{'$1': ''1'', '$2': ''1'', '$3': ''643'', '$4': ''1'', '$5': ''1'', '$6': ''2022-05-19 02:10:30.19'', '$7': ''1929.2926003643217'', '$8': ''wstwxyo yhmubmfli'', '$9': None}"
3461,COMMIT,"{'$1': None, '$2': None, '$3': None, '$4': None, '$5': None, '$6': None, '$7': None, '$8': None, '$9': None}"


In [2]:
import re

In [4]:
# query_template = """
# SELECT COUNT(DISTINCT (S _I_ID)) AS STOCK_COUNT F ROM order_line, stock WH ERE OL_W_ID = $1 AND OL_D_ID = $2 AND OL_O_ID < $3 AND OL_O_ID >= $4 AND S_W_ID = $5 AND S_I_ID = OL_I_ID AND S_QUANTITY < $6
# """

# query_template = 'SELECT C_DISCOUNT, C_LAST, C_CREDIT FROM WHERE customer C_W_ID = $1 AND C_D_ID = $2 AND C_ID = $3'

# query_template = 'UPDATE warehouse SET W_YTD = W_YTD + $1 WHERE W_ID = $2'

# query_template = "UPDATE customer SET C_BALANCE = $1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3, C_DATA = $4 WHERE C_W_ID = $5 AND C_D_ID = $ 6 AND C_ID = $7"

query_template = "SELECT NO_O_ID FROM new_order WHERE NO_D_ID = $1 AND NO_W_ID = $2 ORDER BY NO_O_ID ASC LIMIT $3"

query_template = 'UPDATE customer SET C_BALANCE = -$1, C_YTD_PAYMENT = $2, C_PAYMENT_CNT = $3 WHERE C_W_ID = $4 AND C_D_ID = $5 AND C_ID = $6'

r = re.compile(r"([\S]+) (?:=|>=|<=|>|<) (?:\S+(?: \+ )?)?\$\d+")
print(r.findall(query_template))

# r2 = re.compile(r"(\S+) \$\d+")
# print(r2.findall(query_template))

['C_BALANCE', 'C_YTD_PAYMENT', 'C_PAYMENT_CNT', 'C_W_ID', 'C_D_ID', 'C_ID']


In [33]:
query_template = "INSERT INTO history (H_C_D_ID, H_C_W_ID, H_C_ID, H_D_ID, H_W_ID, H_DATE, H_AMOUNT, H_DATA) VALUES ($1,$2,$3,$4,$5,$6,$7,$8)"

query_template = "INSERT INTO oorder (O_ID , O_D_ID, O_W_ID, O_C_ID, O_ENTRY_D, O_OL_CNT, O_ALL_LOCAL) VALUES ($1, $2, $3, $4, $5, $6, $7)"

r = re.compile(r"\(([^\$]+)\)")
cols = r.findall(query_template)[0]
print(cols)
cols.split(", ")

O_ID , O_D_ID, O_W_ID, O_C_ID, O_ENTRY_D, O_OL_CNT, O_ALL_LOCAL


['O_ID ', 'O_D_ID', 'O_W_ID', 'O_C_ID', 'O_ENTRY_D', 'O_OL_CNT', 'O_ALL_LOCAL']